# Discrete Prior Class

This notebook details how to use the discrete (log) prior class within `GammaBayes` including: required inputs, attributes, and useful functionality. Once other prior classes are added to the repo they will also be detail in this notebook.

In [1]:
import sys
sys.path.append("..")
from gammabayes.prior import discrete_logprior

## Inputs

All of the inputs and the default values for the discrete (log) prior class are shown below.

1. `name='[None]'`,
2. `inputunit=None`, 
3. `logfunction=None`, 
4. `axes=None`, 
5. `axes_names='[None]'`, 
6. `hyperparameter_axes=None`, 
7. `hyperparameter_names='[None]'`,
8. `default_hyperparameter_values=None`, 
9. `logjacob=0`

The `name ` parameter is not a required parameter to instantiate the class, but details the name of the prior that you are using and can be helpful when you have multiple priors floating around.


----

The `inputunit` tells someone handling the class what are the input units. For example if you are working with energy in TeV, and sky position in galactic coordinates you might add, `inputunit='[TeV, deg, deg]'`.

___

The `logfunction` parameter is first essentially required parameter, being the function that actually outputs the log of the prior probability. Depending on what priors one is using the methods to generate this particular function is quite different and is why it is left as a general input to the class rather than having some in-built method to generate the prior from some standard input. 

The format of the inputs to function should be something like `logfunction(axis_1_val, axis_2_val, axis_3_val..., axis_n_val, hyperparameter_1_value, hyperparameter_2_value,...)` and if there are no hyperparameters then the format should just include the normalisation axes (`axis_1_val`, `axis_2_val`,...).

___

The `axes` function is also generally required, it is the range of values that the function can be evaluated at, hence the 'discrete' in the name, it can only be evaluated at the discrete values given. 

If there are multiple axes over which the prior is defined (e.g. log energy, longitude, latitude) then the input must be formatted as a tuple `(axis_1`, `axis_2`, `axis_3,)`. Unfortunately due to `Python`'s genius/stupidity, when one accesses the length of the tuple to specify the number of dimensions if there is only one axis input such as `(axis_1)` if there is no comma following the axis the length of the tuple returns the length of `axis_1`. To avoid this, if you are specifying one axis, then the format should be `(axis_1,)` and `Python` will correctly return 1 as the number of dimensions.
___

The `axes_names` input should be a tuple of strings for the names of each axis. It is not required.
___

The `hyperparameter_axes` should be a tuple containing of combinations of hyperparameters that you wish to evaluate the prior at by default, this can be over-written or blank when using methods of the class that involve the input of hyperparameters. 

e.g. If you have two hyperparameter axes `logmassrange` and `coupling_range` described as numpy arrays `np.linspace(-1,2,31)` and `np.linspace(0,1,11)` respectively the input tuple could be `(*np.meshgrid(logmassrange, couplingrange, indexing='ij),)`.

___

We then have another naming input, `hyperparameter_axes_names` which should be a tuple containing string representations of the names of the axes.

___

`default_hyperparameter_values` is used when constructing a prior array for a single hyperparameter value when no hyperparameter value is given. Default is `(None,)`.

___

The final possible input is `logjacob` and is the natural log of the jacobian used when integrating the prior over all it's axes. If the axes have shapes `(m_1,)`, `(m_2,)`, `(m_3,)`,..., `(m_n,)` then the logjacob argument must be of shape `(m_1, m_2, m_3,..., m_n,)`. 

e.g. You have three axes of $log_{10}$ energy, galactic longitude and galactic latitude. To normalise the prior with respect to energy one must include a jacobian $\propto log_{e}(E)$ with no jacobian needed for the other axes. The `logjacob` input should then be something like `logjacob=np.meshgrid(np.log(10**log10 energy axis), galactic longitude axis, galactic latitude axis)[0]` so that `logjacob` has the right shape.

## Demonstration (attributes and functionality)